# Extraction and Loading in db

In [ ]:
import json
import requests
import pandas as pd

# Pokemon Types

In [35]:
import requests
import pandas as pd
from sqlalchemy import create_engine

def extract_pokemon_types(limit=911):
    url = 'https://pokeapi.co/api/v2/pokemon'
    pokemon_data = []
    count = 0

    while url and count < limit:
        response = requests.get(url)

        if response.status_code == 200:
            pokemon_page = response.json()
            results = pokemon_page['results']

            for pokemon in results:
                if count >= limit:
                    break

                pokemon_name = pokemon['name']
                pokemon_url = pokemon['url']
                pokemon_response = requests.get(pokemon_url)

                if pokemon_response.status_code == 200:
                    pokemon_info = pokemon_response.json()
                    types = [type['type']['name'] for type in pokemon_info['types']]
                    if len(types) > 1:
                        sub_type = types[1]
                    else:
                        sub_type = None

                    pokemon_data.append({'pokemon_name': pokemon_name, 'type': types[0], 'sub_type': sub_type})

                count += 1

            url = pokemon_page['next']
        else:
            print('Error:', response.status_code)
            break

    return pd.DataFrame(pokemon_data)


# Example usage
types_df = extract_pokemon_types(limit=911)


# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Store the DataFrame in PostgreSQL
types_df.to_sql('pokemon_types', engine, if_exists='replace', index=False)



911

# Pokemon Stats

In [38]:

from sqlalchemy import create_engine

def extract_pokemon_stats(limit=911):
    url = 'https://pokeapi.co/api/v2/pokemon'
    pokemon_data = []
    count = 0

    while url and count < limit:
        response = requests.get(url)

        if response.status_code == 200:
            pokemon_page = response.json()
            results = pokemon_page['results']

            for pokemon in results:
                if count >= limit:
                    break

                pokemon_name = pokemon['name']
                pokemon_url = pokemon['url']
                pokemon_response = requests.get(pokemon_url)

                if pokemon_response.status_code == 200:
                    pokemon_info = pokemon_response.json()
                    stats = {stat['stat']['name']: stat['base_stat'] for stat in pokemon_info['stats']}
                    pokemon_data.append({'pokemon_name': pokemon_name, **stats})

                count += 1

            url = pokemon_page['next']
        else:
            print('Error:', response.status_code)
            break

    return pd.DataFrame(pokemon_data)


# Example usage
stats_df = extract_pokemon_stats(limit=911)

# print(stats_df)


# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Store the DataFrame in PostgreSQL
stats_df.to_sql('pokemon_stats', engine, if_exists='replace', index=False)


911

# Pokemon Generations

In [5]:
import requests
import pandas as pd
from sqlalchemy import create_engine

def extract_pokemon_generation(limit=911):
    url = 'https://pokeapi.co/api/v2/pokemon'
    pokemon_data = []
    count = 0

    while url and count < limit:
        response = requests.get(url)

        if response.status_code == 200:
            pokemon_page = response.json()
            results = pokemon_page['results']

            for pokemon in results:
                if count >= limit:
                    break

                pokemon_name = pokemon['name']
                pokemon_url = pokemon['url']
                pokemon_response = requests.get(pokemon_url)

                if pokemon_response.status_code == 200:
                    pokemon_info = pokemon_response.json()
                    species_url = pokemon_info['species']['url']
                    species_response = requests.get(species_url)

                    if species_response.status_code == 200:
                        species_info = species_response.json()
                        generation = species_info['generation']['name']
                        pokemon_data.append({'pokemon_name': pokemon_name, 'generation': generation})

                count += 1

            url = pokemon_page['next']
        else:
            print('Error:', response.status_code)
            break

    return pd.DataFrame(pokemon_data)


# Example usage
gen_df = extract_pokemon_generation(limit=911)

# print(pokemon_df)


# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Store the DataFrame in PostgreSQL
gen_df.to_sql('pokemon_generation', engine, if_exists='replace', index=False)



911

# Pokemon Evalutions

In [23]:
import requests
import pandas as pd

def extract_pokemon_evolution_chain(limit=911):
    url = 'https://pokeapi.co/api/v2/pokemon'
    evolution_chain_data = []
    count = 0

    while url and count < limit:
        response = requests.get(url)

        if response.status_code == 200:
            pokemon_page = response.json()
            results = pokemon_page['results']

            for pokemon in results:
                if count >= limit:
                    break

                pokemon_name = pokemon['name']
                pokemon_url = pokemon['url']
                pokemon_response = requests.get(pokemon_url)

                if pokemon_response.status_code == 200:
                    pokemon_info = pokemon_response.json()
                    species_url = pokemon_info['species']['url']
                    species_response = requests.get(species_url)

                    if species_response.status_code == 200:
                        species_info = species_response.json()
                        evolution_chain_url = species_info['evolution_chain']['url']
                        evolution_chain_response = requests.get(evolution_chain_url)

                        if evolution_chain_response.status_code == 200:
                            evolution_chain_info = evolution_chain_response.json()
                            chain = evolution_chain_info['chain']
                            evolution_stages = extract_evolution_chain_data(chain)
                            evolution_chain_data.append(evolution_stages)

                count += 1

            url = pokemon_page['next']
        else:
            print('Error:', response.status_code)
            break

    return evolution_chain_data


def extract_evolution_chain_data(chain):
    stages = [chain['species']['name']]

    if 'evolves_to' in chain:
        evolves_to = chain['evolves_to']
        for evolution in evolves_to:
            stages.extend(extract_evolution_chain_data(evolution))

    return stages


# Example usage
evolution_chain_data = extract_pokemon_evolution_chain(limit=911)

max_stages = max(len(chain) for chain in evolution_chain_data)
columns = [f'stage_{i}' for i in range(1, max_stages + 1)]

evolution_chain_df = pd.DataFrame(evolution_chain_data, columns=columns)
evolution_chain_df.drop_duplicates(inplace=True)
print(evolution_chain_df)


# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Store the DataFrame in PostgreSQL
evolution_chain_df.to_sql('pokemon_evolution_chart', engine, if_exists='replace', index=False)


        stage_1     stage_2      stage_3 stage_4 stage_5 stage_6 stage_7  \
0     bulbasaur     ivysaur     venusaur    None    None    None    None   
3    charmander  charmeleon    charizard    None    None    None    None   
6      squirtle   wartortle    blastoise    None    None    None    None   
9      caterpie     metapod   butterfree    None    None    None    None   
12       weedle      kakuna     beedrill    None    None    None    None   
..          ...         ...          ...     ...     ...     ...     ...   
896   spectrier        None         None    None    None    None    None   
897     calyrex        None         None    None    None    None    None   
904    enamorus        None         None    None    None    None    None   
905  sprigatito   floragato  meowscarada    None    None    None    None   
908     fuecoco    crocalor   skeledirge    None    None    None    None   

    stage_8 stage_9  
0      None    None  
3      None    None  
6      None    None  

471

In [24]:
from sqlalchemy import create_engine


# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Execute the SQL query and store the result in a DataFrame
query = "SELECT stage_1 FROM pokemon_evolution_chart WHERE stage_2 IS NULL"
result_df = pd.read_sql(query, engine)

# Store the DataFrame as a new table in PostgreSQL
result_df.to_sql('pokemon_stage_1', engine, if_exists='replace', index=False)

161

In [27]:
from sqlalchemy import create_engine

# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Execute the SQL query and store the result in a DataFrame
query="select stage_1,stage_2 from pokemon_evolution_chart where stage_3 is null and stage_2 is not null union all \
select stage_1,concat(stage_2,', ',stage_3,', ',stage_4,', ',stage_5,', ',stage_6,', ',stage_7,', ',stage_8,', ',stage_9) \
as stage_2 from pokemon_evolution_chart where stage_1 is not null and stage_2 is not null and stage_3 is not null \
and  stage_4 is not null and stage_5 is not null and stage_6 is not null and stage_7 is not null and \
stage_8 is not null and  stage_9 is not null"
result_df = pd.read_sql(query, engine)

# Store the DataFrame as a new table in PostgreSQL
result_df.to_sql('pokemon_stage_2', engine, if_exists='replace', index=False)

189

In [28]:
from sqlalchemy import create_engine

# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Execute the SQL query and store the result in a DataFrame
query="select stage_1,stage_2,stage_3 from pokemon_evolution_chart \
where stage_4 is null and stage_2 is not null and stage_3 is not null"
result_df = pd.read_sql(query, engine)

# Store the DataFrame as a new table in PostgreSQL
result_df.to_sql('pokemon_stage_3', engine, if_exists='replace', index=False)

115

In [29]:
from sqlalchemy import create_engine


# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'
# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


# Execute the SQL query and store the result in a DataFrame
query="select stage_1,stage_2,stage_3,stage_4 from pokemon_evolution_chart \
where stage_5 is null and stage_2 is not null and stage_3 is not null and stage_4 is not null"
result_df = pd.read_sql(query, engine)

# Store the DataFrame as a new table in PostgreSQL
result_df.to_sql('pokemon_stage_4', engine, if_exists='replace', index=False)

5

# Pokemon Images

In [19]:
import requests
import pandas as pd
from sqlalchemy import create_engine


def extract_pokemon_images(limit=911):
    url = 'https://pokeapi.co/api/v2/pokemon'
    pokemon_data = []
    count = 0

    while url and count < limit:
        response = requests.get(url)

        if response.status_code == 200:
            pokemon_page = response.json()
            results = pokemon_page['results']

            for pokemon in results:
                if count >= limit:
                    break

                pokemon_name = pokemon['name']
                pokemon_url = pokemon['url']
                pokemon_response = requests.get(pokemon_url)

                if pokemon_response.status_code == 200:
                    pokemon_info = pokemon_response.json()
                    image_url = pokemon_info['sprites']['front_default']
                    pokemon_data.append({'pokemon_name': pokemon_name, 'image': image_url})

                count += 1

            url = pokemon_page['next']
        else:
            print('Error:', response.status_code)
            break

    return pd.DataFrame(pokemon_data)


# Example usage
pokemon_image_df = extract_pokemon_images(limit=911)

# PostgreSQL connection details
db_host = 'yourhost'
db_port = 'your_port'
db_name = 'your_dbname'
db_user = 'your_username'
db_password = 'your_password'

# Create the SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Store the DataFrame in PostgreSQL
pokemon_image_df.to_sql('pokemon_image', engine, if_exists='replace', index=False)


911